In [1]:
import tensorflow as tf
import gym
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, ReLU
from tensorflow.keras.optimizers import Adam
import time
import scipy.signal
from tqdm import tqdm

Init Plugin
Init Graph Optimizer
Init Kernel


# TrajectoryStorage


In [2]:
class Storage:
    def __init__(self):
        # init creating a storage for all observation variables during trajectory
        self.observations = []
        # create arrays for chosen actions and rewards
        self.actions = []
        self.logits = []
        self.rewards = []
        self.BaselineEstimate = []
        # finished episodes will be completely stored in this list 
        self.episodes = []


    def store(self,observation, action, logits, reward, BaselineEstimate):
        self.observations.append(observation)
        self.actions.append(action)
        self.logits.append(logits)
        self.rewards.append(reward)
        self.BaselineEstimate.append(BaselineEstimate) # value of critics network
        

    def conclude_episode(self):
        # append all already stored values to finished episodes
        self.episodes.append(
            [self.observations,
             self.actions, 
             self.logits,
             self.rewards,
             self.BaselineEstimate, 
             sum(self.rewards)]) # get the return of the whole episode
             
        # empty the arrays for new trajectory
        self.observations.clear()
        # create arrays for chosen actions and rewards
        self.actions.clear()
        self.logits.clear()
        self.rewards.clear()
        self.BaselineEstimate.clear()


    # return array of finished trajectories stored in self.episodes and the amount of episodes
    def get_episodes(self):
        return self.episodes, len(self.episodes)
        
        

# Actor Model


In [3]:
class Actor(Model):
    def __init__(self):
        super(Actor, self).__init__()


        self.l = [
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(4, activation="softmax")
        ]

    #@tf.function        
    def call(self, x):
        for l in self.l:
            x = l(x)
        return x

        

#@tf.function
def sample_action(observation):
    logits = actor(observation)
   # tf.print(type(logits))
    action = tf.squeeze(tf.random.categorical(logits, 1), axis=1)
   # tf.print(action)
    return logits, action

# Critic Model

In [4]:
class Critic(Model):
    def __init__(self):
        super(Critic, self).__init__()

        self.l = [
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(1, activation="tanh", kernel_regularizer=tf.random_normal_initializer(stddev=0.01))
        ]

    #@tf.function 
    def call(self, x):
        for l in self.l:
            x = l(x)
        return x

In [5]:
# define Hyperparameters
epochs = 1
steps_per_epoch = 1000 # ~10 Episodes per epoch, then compute new parameters (smaller batching)
lr_actor = 3e-4
lr_critic = 3e-4
train_policy_iterations = 80
train_value_iterations = 80
clip_ratio = 0.2
target_kl = 0.01
optimizer = Adam()

render = False

In [6]:
tf.keras.backend.clear_session()

# define environment
env = gym.make("LunarLander-v2")
# get observation_dims and amount of possible actions (1 for CartPole-v1)
observation_dimensions = env.observation_space.shape[0]
num_actions = env.action_space.n

# create Storage for observations, actions, rewards etc during trajectory
T = Storage()

# init the actor and critics model
observation_input = Input(shape=(observation_dimensions,), dtype=tf.float32)
actor = Actor()
critic = Critic()

# Initialize the observation, episode return and episode length
observation, episode_return, episode_length = env.reset(), 0, 0

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-04 12:37:46.563900: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-04 12:37:46.564041: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
episodes_total = 0
for epoch in range(epochs):

    sum_return = 0
    sum_length = 0
    num_episodes = 0

    for t in tqdm(range(steps_per_epoch)):
        if render:
            env.render()

        observation = observation.reshape(1,-1)

        # obtain action and logits for this observation by our actor
        logits, action = sample_action(observation=observation)
        
        # make a step in environment and obtain the rewards for it
        observation_new, reward, done, _ = env.step(action[0].numpy())

        # sum up rewards over this episode and count amount of frames
        episode_return += reward
        episode_length += 1

        # get the Base-Estimate from the Critics network
        base_estimate = critic(observation)

        # store Variables collected in this step
        T.store(observation=observation, action=action, logits=logits, reward=reward, BaselineEstimate=base_estimate)
        #update the observations
        observation = observation_new
        # check if terminal state is reached in env, if so save episode and refresh storage, reset env
        if done:
            T.conclude_episode()
            observation, episode_return, episode_length = env.reset(), 0, 0

    # obtain all episodes saved in storage
    episodes, amount_episodes = T.get_episodes()

  

        


100%|██████████| 1000/1000 [00:03<00:00, 289.34it/s]


In [8]:
# episodes [episode][particular values // 0: Observations, 1: actions, 2: logits, 3, rewards, 4: BaselineEstimates from Critics]
#print(episodes[0][4])
print(f'Number of Episodes = {amount_episodes}')



### Advantagefunction

# estimated Value of the current situtation from the critics network
b_estimates = episodes[0][4] 
# for i in b_estimates:
#   print(i.numpy())

# Discounted sum of rewards
print(episodes[0][3]) 
rewards = episodes[0][3]
gamma = 0.99

    



Number of Episodes = 11
[-0.4264440701853971, -1.5051301517936964, -0.8757260133700584, -0.5711668046340606, -0.6118232836923028, -1.6563027111441204, -2.238994327738594, -0.06036102388392692, -2.4638427059447165, -0.9674875484260212, -1.989685212016069, -0.042337950427308896, -2.5407700434440685, -2.672685094051586, -0.6773556384971176, -1.9671823165295337, -0.11061789709731215, -0.3620920355987767, -0.06086519084712336, -2.2888636484981633, -0.17861734669699444, 0.12173367041177244, -0.9404595347056954, -2.3109151863718, -1.5396380127457292, -2.6744072578578355, -0.7022746271788833, -0.58683251490251, -2.5657968516759113, -1.587873237786198, -0.3701425921136934, -0.31221755825171615, 0.0735138751271063, -1.0249967457178002, -1.9926045140670385, -0.024850375548622877, -1.003259948603727, -0.02721960076485061, 0.24028436410896006, -0.6333837462898089, -1.9373512580493457, -1.2214220652484926, -0.7644795585776194, -1.8295434699770567, -1.63987780976355, -0.5757665794678815, -0.572837325

In [9]:
def discounted_reward(rewards, gamma):
    i = 0
    discounted_rewards = []
    for r in rewards:
        disc = 0
        for t in rewards[i:-1]:
            discount_t = gamma ** t
            disc += t * discount_t
        i += 1
        discounted_rewards.append(disc)
    return discounted_rewards
    
    


In [10]:
# get discounted sum of rewards 
disc_sum = discounted_reward(rewards, gamma)

# convert lists to np arrays and flatten
disc_sum_np = np.array(disc_sum)
b_estimates_np = np.array(b_estimates)
b_estimates_np = b_estimates_np.flatten()

# substract arrays to obtain advantages
advantages = np.subtract(disc_sum_np, b_estimates_np)
print(advantages)





[-3.33844649e+01 -3.29561888e+01 -3.14281182e+01 -3.05446514e+01
 -2.99701960e+01 -2.93545985e+01 -2.76704920e+01 -2.53805426e+01
 -2.53201444e+01 -2.27945299e+01 -2.18175885e+01 -1.97877132e+01
 -1.97453579e+01 -1.71388695e+01 -1.43934193e+01 -1.37114347e+01
 -1.17049739e+01 -1.15942338e+01 -1.12308213e+01 -1.11699179e+01
 -8.82778895e+00 -8.64884978e+00 -8.77043395e+00 -7.82104144e+00
 -5.45582340e+00 -3.89217724e+00 -1.14490734e+00 -4.37659831e-01
  1.52645697e-01  2.78546998e+00  4.39888832e+00  4.77041108e+00
  5.08361042e+00  5.01015137e+00  6.04576325e+00  8.07867514e+00
  8.10353345e+00  9.11696173e+00  9.14419087e+00  8.90448750e+00
  9.54191713e+00  1.15173614e+01  1.27538679e+01  1.35242456e+01
  1.53877402e+01  1.70548694e+01  1.76339792e+01  1.82101259e+01
  1.98827011e+01  2.06428202e+01  2.09197574e+01  2.04551855e+01
  2.03543975e+01  1.96966149e+01  1.91313666e+01  1.74938412e+01
  1.65271210e+01  1.65135034e+01  1.76881785e+01  1.70963082e+01
  1.71395228e+01  1.69515

# LogProbs and Ratio Computation

We need the ratio of probabilities for an action at state t of the 'new' model vs the old model (maybe because of entropy there is a difference)

In [64]:

#creating oneHot vector with size actions space and getting the log for the probability of choosing this action

# print(f'log old: {logits_old}')
# print(f'log new: {logits_new}')


### this function currently only takes one single action and 2 sets of logits and computes the ratio of that

# def get_ratio(action, logits_old, logits_new):

#     #get the Logarithmic version of all logits for computational efficiency
#     log_prob_old = tf.nn.log_softmax(logits_old)
#     log_prob_new = tf.nn.log_softmax(logits_new)

#     # encode in OneHotVector and reduce to sum, giving the log_prob for the action the agent took for both policies
#     logprobability_old = tf.reduce_sum(
#         tf.one_hot(action, num_actions) * log_prob_old, axis=1
#     )
#     logprobability_new = tf.reduce_sum(
#         tf.one_hot(action, num_actions) * log_prob_new, axis=1
#     )
#     # get the ratio of new over old prob
#     ratio = tf.exp(logprobability_new - logprobability_old)

#     print(ratio)



def get_ratio_episode(actions, logits_old, logits_new):
    r = []
    for a, o, n in zip(actions, logits_old, logits_new):
        #get the Logarithmic version of all logits for computational efficiency
        log_prob_old = tf.nn.log_softmax(o)
        log_prob_new = tf.nn.log_softmax(n)

        # encode in OneHotVector and reduce to sum, giving the log_prob for the action the agent took for both policies
        logprobability_old = tf.reduce_sum(
            tf.one_hot(a, num_actions) * log_prob_old, axis=1
        )
        logprobability_new = tf.reduce_sum(
            tf.one_hot(a, num_actions) * log_prob_new, axis=1
        )
        # get the ratio of new over old prob
        ratio = tf.exp(logprobability_new - logprobability_old)
        r.append(ratio)
    return r
        


## second test with multiple actions taken by the agent
obs = episodes[0][0]
logits_new = []
for i in obs:
    tensor = tf.convert_to_tensor(i)
    new, _ = sample_action(tensor)
    logits_new.append(new) 

episode_actions = episodes[0][1]
episode_logits_old = episodes[0][2]
# print(len(logits_new), len(episode_logits_old), len(episode_actions))


# episode_1 = get_ratio_episode(episode_actions, episode_logits_old, logits_new)
# print(len(episode_1))
# print(f'Length fo Episode: {len(episode_1)}. Episode Ratios: {episode_1}')

batch_obs = episodes[:][0]
print(type(batch_obs), len(batch_obs))
nones = np.full_like(batch_obs,None)
#print(nones)

array_ops =  np.array(batch_obs)
for i in range(len(array_ops)):
    nones[i] = episodes[i][0]
print(type(nones))

array_ops = np.array(array_ops)
#print(f'Obs_episode_array: {array_ops}')
def get_ratio_batch(actions, logits_old, logits_new):
    print('bla')


<class 'list'> 6
<class 'numpy.ndarray'>


/var/folders/jk/h_92czjx5jjcf631wf82ht9c0000gn/T/ipykernel_3107/2711087504.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_ops =  np.array(batch_obs)
